In [12]:
from artm_experiments import Experiment, Pool, GreedyTopicsFilter
from artm import *
import glob
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
batch_vectorizer = BatchVectorizer(data_path='', data_format='bow_uci', collection_name='kos', target_folder='kos')

model_artm = ARTM(num_topics=15,
                  scores=[PerplexityScore(name='PerplexityScore',
                                               use_unigram_document_model=False,
                                               dictionary_name='dictionary')],
                  regularizers=[SmoothSparseThetaRegularizer(name='SparseTheta', tau=-0.2)])

#model_artm.gather_dictionary(dictionary_target_name='dictionary', 
#                             data_path='kos', vocab_file_path='vocab.kos.txt')
#model_artm.save_dictionary(dictionary_name='dictionary', dictionary_path='kos/dictionary.dict')
model_artm.load_dictionary(dictionary_name='dictionary', dictionary_path='kos/dictionary.dict')
model_artm.initialize(dictionary_name='dictionary')

model_artm.regularizers.add(SmoothSparsePhiRegularizer(name='SparsePhi', tau=-0.2))
model_artm.regularizers.add(DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=2.5e+5))

In [25]:
exp = Experiment([{'model': model_artm, 'factor': 5}], Pool(topics_filter=GreedyTopicsFilter(sigma=5)))
exp.load_data('kos')
exp.run()

Total basic topics: 15


In [26]:
exp.show_next_topics_batch(15)

topic0:
[u'governor' u'electoral' u'account' u'november' u'house']
topic1:
[u'dean' u'edwards' u'clark' u'primary' u'democratic']
topic2:
[u'qaqaa' u'parecommend' u'labor' u'ticket' u'north']
topic3:
[u'voters' u'poll' u'polls' u'percent' u'results']
topic4:
[u'iraq' u'war' u'iraqi' u'american' u'saddam']
topic5:
[u'republicans' u'senate' u'republican' u'gop' u'democrats']
topic6:
[u'white' u'president' u'jobs' u'administration' u'years']
topic7:
[u'sunday' u'campaign' u'media' u'ads' u'news']
topic8:
[u'house' u'delay' u'committee' u'money' u'million']
topic9:
[u'election' u'state' u'states' u'ohio' u'voter']
topic10:
[u'democrats' u'race' u'candidates' u'state' u'district']
topic11:
[u'political' u'party' u'tax' u'issue' u'vote']
topic12:
[u'military' u'service' u'national' u'white' u'guard']
topic13:
[u'john' u'president' u'general' u'bushs' u'speech']
topic14:
[u'political' u'people' u'time' u'media' u'news']


In [28]:
exp.save_dataset_to_navigator()

In [29]:
exp.save_next_topics_to_navigator()